In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
raw_original_df = pd.read_csv('2020_LoL_esports_match_data_from_OraclesElixir_20201016.csv')
teams_df = pd.read_csv('teams_df_final.csv')
raw_worlds_df = pd.read_csv('worlds_raw_df.csv')

## Voy a extraer los nombres de todos los equipos que participaron en el campeonato mundial, a partir de la lista obtenida haremos el desempeño promedio de los equipos, ya que son los unicos que nos interesan

In [4]:
worlds_only_teams = raw_worlds_df[raw_worlds_df['league'] == 'WCS']

In [5]:
worlds_only_teams = worlds_only_teams[worlds_only_teams['position'] == 'team']

In [58]:
i = 0
matches = []
while i < len(worlds_only_teams):
    temp_match = [worlds_only_teams.iloc[i]['team'], worlds_only_teams.iloc[i+1]['team'], worlds_only_teams.iloc[i]['result']]
    matches.append(temp_match)
    #print(worlds_only_teams.iloc[i]['team'], 'VS.', worlds_only_teams.iloc[i+1]['team'], '| winner: ', worlds_only_teams.iloc[i]['result'])
    i += 2

In [61]:
matches_df = pd.DataFrame(matches, columns = ['blue_team', 'red_team', 'result'])
#matches_df

,blue_team,red_team,result
0,Suning,LGD Gaming,1
1,LGD Gaming,Suning,0
2,LGD Gaming,Suning,0
3,FunPlus Phoenix,Invictus Gaming,1
4,FunPlus Phoenix,Invictus Gaming,0
...,...,...,...
131,Suning,Top Esports,1
132,DAMWON Gaming,Suning,1
133,DAMWON Gaming,Suning,0
134,Suning,DAMWON Gaming,0


In [62]:
#matches_df.to_csv('worlds_matches.csv')

In [6]:
#worlds_only_teams.info()

In [7]:
worlds_teams = worlds_only_teams['team'].unique()

In [8]:
#worlds_teams

In [9]:
worlds_teams_purged = []
for team in worlds_teams:
    if team != 'Invictus Gaming' and team != 'Afreeca Freecs' and team != 'KT Rolster':
        if team == 'TSM':
            worlds_teams_purged.append('Team SoloMid')
        else:
            worlds_teams_purged.append(team)

In [10]:
#worlds_teams_pruged, len(worlds_teams_pruged)

## A partir de este punto voy a inicializar el proceso para promediar los resultados de los equipos, genere un nuevo dataframe "team_df_names.csv" es identico a teams_df_final pero incluye los nombres de los equipos en las partidas

In [11]:
teams_names = pd.read_csv('team_df_names.csv')


In [12]:
opp_col = ['golddiffat10', 'xpdiffat10', 'csdiffat10', 'golddiffat15', 'xpdiffat15', 'csdiffat15']
col = []

col_names = teams_names.columns

In [13]:
for column in col_names:
    if column[:4] == 'opp_':
        opp_col.append(column)
    else:
        col.append(column)

In [14]:
blue_teams = teams_names[col]

In [15]:
#blue_teams

In [16]:
red_teams = teams_names[opp_col]
#red_teams

In [17]:
blue_teams.drop('gameid', inplace = True, axis = 1)

C:\Users\kesha\anaconda3\envs\ml\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
blue_teams.drop('gamelength', inplace = True, axis = 1)
blue_teams.drop('result', inplace = True, axis = 1)

In [21]:
red_cols_fix = []
rc = red_teams.columns
for cols in rc:
    if cols[4:] == 'team_kpm':
        red_cols_fix.append('team kpm')
    elif cols[4:] == 'earned_gpm':
        red_cols_fix.append('earned gpm')
    elif cols[:4] == 'opp_':
        red_cols_fix.append(cols[4:])
    else:
        red_cols_fix.append(cols)
    
red_teams.columns = red_cols_fix

In [25]:
blue_columns = blue_teams.columns
red_columns = red_teams.columns

In [40]:
worlds_avg_df = pd.DataFrame(columns = blue_columns)

In [41]:
worlds_avg_df

,team,kills,deaths,assists,doublekills,triplekills,quadrakills,pentakills,firstblood,team kpm,ckpm,firstdragon,dragons,elementaldrakes,infernals,mountains,clouds,oceans,elders,firstherald,heralds,firstbaron,barons,firsttower,towers,firstmidtower,firsttothreetowers,inhibitors,damagetochampions,dpm,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,goldspent,gspd,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,golddiffat10,xpdiffat10,csdiffat10,goldat15,xpat15,csat15,golddiffat15,xpdiffat15,csdiffat15


In [42]:
for team in worlds_teams_purged:
    team_blue = blue_teams[blue_teams['team'] == team]
    team_red = red_teams[red_teams['team'] == team]
    
    list_dataframe = []
    nan = [np.nan for x in range(len(team_red))]
    for cols in blue_columns:
        if cols in red_columns:
            list_dataframe.append(team_red[cols].values)
        else:
            list_dataframe.append(nan)
            
    transpose_df = []
    for i in range(len(list_dataframe[0])):
        temp = []
        for j in range(len(list_dataframe)):
            temp.append(list_dataframe[j][i])
        transpose_df.append(temp)
        
    append_df = pd.DataFrame(transpose_df, columns = blue_columns)
    
    team_complete = team_blue.append(append_df)
    
    team_average = team_complete.mean().values
    
    team_average_fix = [team]
    for x in team_average:
        team_average_fix.append(x)
    
    team_average_fix = [team_average_fix]
    
    team_average_df = pd.DataFrame(team_average_fix, columns = blue_columns)
    
    worlds_avg_df = worlds_avg_df.append(team_average_df)

In [43]:
worlds_avg_df

,team,kills,deaths,assists,doublekills,triplekills,quadrakills,pentakills,firstblood,team kpm,ckpm,firstdragon,dragons,elementaldrakes,infernals,mountains,clouds,oceans,elders,firstherald,heralds,firstbaron,barons,firsttower,towers,firstmidtower,firsttothreetowers,inhibitors,damagetochampions,dpm,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,goldspent,gspd,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,golddiffat10,xpdiffat10,csdiffat10,goldat15,xpat15,csat15,golddiffat15,xpdiffat15,csdiffat15
0,Suning,16.285714,12.000000,35.000000,1.000000,0.142857,0.142857,0.000000,0.571429,0.497557,0.872875,0.500000,2.857143,2.714286,1.000000,0.428571,0.428571,0.857143,0.142857,0.285714,0.857143,0.428571,1.000000,0.428571,8.142857,0.857143,0.857143,1.428571,76299.142857,2230.155543,3000.768443,2843.757843,138.714286,4.135100,63.428571,1.813614,43.142857,301.000000,8.808386,64304.142857,42284.285714,1259.060929,58790.285714,0.058903,901.000000,265.428571,156.857143,42.571429,34.699171,15814.857143,18873.142857,333.428571,-505.000000,-320.142857,-0.428571,25363.857143,30987.428571,543.571429,-914.857143,-401.428571,-0.714286
0,LGD Gaming,13.823529,12.000000,32.352941,1.588235,0.411765,0.058824,0.000000,0.705882,0.407271,0.713600,0.571429,2.647059,2.588235,0.764706,0.823529,0.588235,0.411765,0.058824,0.411765,1.058824,0.470588,0.588235,0.588235,7.411765,0.764706,0.764706,1.235294,66169.529412,1902.670229,2697.978829,2579.890324,119.294118,3.436665,61.470588,1.760753,38.764706,277.823529,8.036965,61806.764706,39473.117647,1160.542200,57763.352941,0.053452,889.000000,250.176471,156.764706,29.058824,33.391012,15557.764706,18278.352941,323.000000,-268.117647,-81.470588,0.352941,24796.294118,29908.588235,523.882353,-154.117647,-186.823529,-8.882353
0,FunPlus Phoenix,14.363636,13.363636,33.090909,1.545455,0.000000,0.000000,0.000000,0.454545,0.412855,0.913600,0.500000,2.545455,2.272727,0.636364,0.636364,0.636364,0.363636,0.272727,0.545455,0.909091,0.272727,0.545455,0.454545,5.909091,0.454545,0.454545,0.909091,63815.363636,1879.614064,3137.386255,3005.412082,104.363636,3.089509,46.545455,1.357791,46.090909,226.909091,6.633427,59358.363636,37489.454545,1106.682364,55017.909091,-0.098576,831.363636,215.181818,145.454545,18.272727,31.363164,15692.454545,18038.090909,309.909091,-388.636364,-510.181818,-8.181818,25484.272727,29294.909091,497.181818,-987.181818,-1179.818182,-11.272727
0,T1,12.724771,8.688073,29.990826,1.605505,0.302752,0.055046,0.000000,0.559633,0.395049,0.628842,0.543860,2.596330,2.541284,0.633028,0.633028,0.623853,0.651376,0.055046,0.449541,0.963303,0.513761,0.678899,0.522936,6.889908,0.550459,0.532110,1.128440,60212.302752,1790.000999,2546.927983,2569.055961,134.908257,3.986352,63.036697,1.843652,48.963303,293.110092,8.638594,60627.798165,38733.110092,1165.820856,56104.825688,0.063855,914.155963,240.174312,156.697248,22.871560,34.556922,15572.348624,18505.201835,331.990826,179.467890,56.954128,5.880734,24677.495413,29808.917431,533.045872,375.532110,-45.321101,4.155963
0,Gen.G,13.864865,9.387387,31.513514,1.765766,0.414414,0.090090,0.027027,0.648649,0.443237,0.738281,0.592593,2.927928,2.855856,0.675676,0.711712,0.792793,0.675676,0.072072,0.630631,1.162162,0.612613,0.729730,0.630631,7.369369,0.666667,0.693694,1.207207,60936.225225,1880.814068,2605.501413,2499.764721,123.108108,3.770559,58.558559,1.787704,48.531532,265.198198,8.135200,58918.135135,37762.477477,1179.495122,53942.198198,0.074497,871.225225,209.945946,130.945946,20.828829,33.582157,15826.234234,18411.945946,326.072072,200.792793,174.180180,6.702703,25051.018018,29637.261261,523.180180,581.720721,199.225225,5.567568
0,MAD Lions,13.794118,14.132353,31.411765,1.382353,0.235294,0.014706,0.000000,0.485294,0.424466,0.847970,0.407407,2.558824,2.441176,0.558824,0.588235,0.558824,0.735294,0.117647,0.573529,1.000000,0.470588,0.676471,0

In [45]:
#worlds_avg_df.to_csv('worlds_avg_df.csv')